In [3]:
import wandb

api = wandb.Api(overrides={"base_url": "https://microsoft-research.wandb.io/"})
#run = api.run("/msr-mtl/text-games-benchmark/runs/7e1kyl3h")
runs = api.runs(
    "msr-mtl/text-games-benchmark",
    filters={
        "state": "finished",
        "config.model": {"$in": [
            "gpt-4o",
            "phi-3-medium-128k-instruct-1",
            "microsoft/Phi-3.5-mini-instruct",
            "meta-llama-3-70b-instruct-4",
            "microsoft/Phi-3-mini-128k-instruct",
            "random-agent",
        ]}
    }
)

#print(run.history())
print(len(runs))


321


In [1]:
import wandb

api = wandb.Api(overrides={"base_url": "https://microsoft-research.wandb.io/"})
runs = api.runs(
    "msr-mtl/text-games-benchmark",
    filters={
        "createdAt": {"$gte": "2024-10-12"},
        "config.framework": {"$eq": "jericho"},
    }
)

#print(run.history())
print(len(runs))

166


In [2]:
print(runs[0].config)
print(runs[0].summary)

{'llm': 'llama-32-3b-instruct', 'game': 'JerichoEnv905', 'seed': 20241001, 'agent': 'LLMAgent_llama-32-3b-instruct_s20241001_c100_t0.0_convTrue', 'context': 100, 'act_temp': 0, 'framework': 'jericho', 'max_steps': 1000, 'conversation': True, 'admissible_commands': False}
{'_runtime': 1789.889927197, '_step': 1001, '_timestamp': 1729129949.519147, '_wandb': {'runtime': 1789}, 'episode/moves': 8, 'episode/normalized_score': 0, 'episode/rollout': {'_latest_artifact_path': 'wandb-client-artifact://grsu3qsr7ni184o68ajlhk76autm9mg3dhch020symlvwvz0tdwhmyw1ggp7g9w25rbg2flakakak8lpztc7jbv6lxsaj3le461jrjg0r54ba27erod3dfj5tb0lj9sp:latest/episode/rollout.table.json', '_type': 'table-file', 'artifact_path': 'wandb-client-artifact://jbdjvqrqvdnjpryzqjvlz0uu746dpx4fp3vrhho2y7pd86fxo3o6ba68lfr78d9s1xpsue27im86l2hujsdyf9poil49sy4zq5zt1k67m8r3ysazk1qunmx7lllctpmt/episode/rollout.table.json', 'ncols': 11, 'nrows': 1000, 'path': 'media/table/episode/rollout_1001_eccece55cd8b8976a19c.table.json', 'sha256':

In [1]:
# Load game_infos.tsv
import pandas as pd
game_infos = pd.read_csv("game_infos.tsv", sep="\t")
game_infos

,EnvName,Framework,Filename,Template Action,Solution,Avg. Steps,Stochastic,Dialog,Darkness,Nonstandard Action,...,Max Score,Maze,Trivia,Self-Maintenance,Combat,Time,Mechanical Puzzle,Persistence,Stochasticity,Forgiveness
0,JerichoEnv905 ...,jericho,advent,107,277,7,TRUE,FALSE,TRUE,TRUE,...,350,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,FALSE,Tough
1,JerichoEnvAcorncourt ...,jericho,balances,30,122,12,FALSE,FALSE,TRUE,TRUE,...,51,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,Polite
2,JerichoEnvAdvent ...,jericho,deephome,93,327,6,TRUE,TRUE,FALSE,TRUE,...,300,TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE,FALSE,Polite
3,JerichoEnvAdventureland ...,jericho,enter,32,102,5,FALSE,TRUE,FALSE,FALSE,...,10,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,Merciful
4,JerichoEnvAfflicted ...,jericho,gold,97,345,10,TRUE,TRUE,FALSE,FALSE,...,100,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,Polite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,TWCookingLevel6 ...,textworld,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,TWCookingLevel7 ...,textworld,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,TWCookingLevel8 ...,textworld,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,TWCookingLevel9 ...,textworld,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Conver Wandb RUNS to a pandas dataframe
import pandas as pd
import numpy as np
runs_data = []
for run in runs:
    run_data = dict(run.summary)
    run_data["agent"] = run.config["agent"]
    run_data["game"] = run.config["game"]
    #run_data["game_description"] = game_infos[game_infos["game"] == run.config["game"]]["description"].values[0]
    runs_data.append(run_data)

runs_df = pd.DataFrame(runs_data)

categories = ["Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory", "Maze", "Maze", "Trivia", "Self-Maintenance", "Combat", "Time", "Mechanical Puzzle", "Persistence", "Difficulty"]
ignore_categories = ["Difficulty"]

# Keep game, model, Max Normalized Score
runs_df = runs_df[["game", "agent", "final/Normalized Score"]]
runs_df = runs_df.sort_values(by="game")

# For each game add columns with the following games infos: Stochastic, Dialog, Darkness, Nonstandard Action, Inventory, Difficulty
for game in runs_df["game"].unique():
    game_info = game_infos[game_infos["EnvName"] == game]
    for col in categories:
        runs_df.loc[runs_df["game"] == game, col] = game_info[col].values[0]

runs_df


NameError: name 'runs' is not defined

In [5]:
# Compute the average score for binary features: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
# and augment the dataframe with these values

radar_df = pd.DataFrame()

for col in categories:
    if col in ignore_categories:
        continue
    radar_df[col] = runs_df.groupby(["agent", col])["final/Normalized Score"].mean().unstack()[True]

radar_df.reset_index(inplace=True)

# Convert to three columns format: agent, feature, score
radar_df = radar_df.melt(id_vars=["agent"], var_name="feature", value_name="score")
radar_df

,agent,feature,score
0,LLMAgent_gpt-4o_s20241001_c100_t0.0_convTrue,Stochastic,0.057790
1,LLMAgent_llama-32-3b-instruct_s20241001_c100_t...,Stochastic,0.012411
2,RandomAgent_s20241001,Stochastic,0.020005
3,LLMAgent_gpt-4o_s20241001_c100_t0.0_convTrue,Dialog,0.048408
4,LLMAgent_llama-32-3b-instruct_s20241001_c100_t...,Dialog,0.011340
5,RandomAgent_s20241001,Dialog,0.050476
6,LLMAgent_gpt-4o_s20241001_c100_t0.0_convTrue,Darkness,0.049306
7,LLMAgent_llama-32-3b-instruct_s20241001_c100_t...,Darkness,0.019195
8,RandomAgent_s20241001,Darkness,0.037659
9,LLMAgent_gpt-4o_s20241001_c100_t0.0_convTrue,Nonstandard Action,0.070721


In [10]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go


fig = px.line_polar(radar_df,
                    r="score",
                    theta="feature",
                    line_group="agent",
                    color="agent",
                    line_close=True,
                    #template="plotly_dark",
                    title="Normalized Score per Criteria",)
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 1])))

# Move legend to the bottom left
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
# Hide title
fig.update_layout(title="")
# Hide legen title
fig.update_layout(legend_title="")
# Change figure size to be
fig.write_image("radar_chart.png", width=1600, height=1600)
fig.show()

fig.update_layout(polar=dict(radialaxis=dict(range=[0, 0.14])))
# Save image to disk
fig.write_image("radar_chart_zoom.png", width=800, height=800)


In [11]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go


fig = px.line_polar(radar_df,
                    r="score",
                    theta="feature",
                    line_group="agent",
                    color="agent",
                    line_close=True, template="plotly_dark", title="Normalized Score per game")
#fig.update_layout(polar=dict(radialaxis=dict(range=[0, 1])))
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 0.14])))
fig.show()

In [189]:
# Using plotly build radar chart for each binary feature: "Stochastic", "Dialog", "Darkness", "Nonstandard Action", "Inventory"
import plotly.express as px
import plotly.graph_objects as go

# Sort runs_df theta according to gpt-4o best max normalized score
runs_df = runs_df.sort_values(by=["model", "Max Normalized Score"], ascending=True)

fig = px.line_polar(runs_df,
                    r="Max Normalized Score",
                    theta="game",
                    line_group="model",
                    color="model",
                    line_close=True, template="plotly_dark", title="Max Normalized Score per game")
fig.update_layout(polar=dict(radialaxis=dict(range=[0, 100])))

fig.show()

In [14]:
# Create a group bar chart with the average score per game for each model
import plotly.express as px
import plotly.graph_objects as go

df = runs_df.copy()

# sort by max score for gpt-4o
#df = df.sort_values(by=["model", "Max Normalized Score"], ascending=True)
# Strip JerichoEnv from game name
df["game"] = df["game"].map(lambda x: x.split("JerichoEnv")[1])

# Filterout game lurking and hollywood
#df = df[~df["game"].isin(["lurking", "hollywood"])]

# Add new column to order difficulty
df["order"] = df["Difficulty"].map({"Possible": 2, "Difficult": 1, "Extreme": 0})

df = df.sort_values(by=["order", "agent", "final/Normalized Score"], ascending=True)

# Add new column color for each game based on the game difficulty
def color(game, diff):
    if diff == "Possible":
        return f"<span style='color:green'>{game}</span>"
    elif diff == "Difficult":
        return f"<span style='color:orange'>{game}</span>"
    elif diff == "Extreme":
        return f"<span style='color:red'>{game}</span>"
    else:
        raise ValueError("Unknown difficulty")

df["color"] = df.apply(lambda x: color(x["game"], x["Difficulty"]), axis=1)


fig = px.bar(df, x="final/Normalized Score", y="game", color="agent", barmode="group", title="Normalized Score per game", height=1000)

# Sort the y axis by game difficulty and then by max score for gpt-4o
#fig.update_yaxes(categoryorder="array", categoryarray=df.sort_values(by=["order", "agent", "final/Normalized Score"], ascending=True))
fig.update_yaxes(tickvals=df["game"], tickmode="array", ticktext=df["color"])

# Make x axis range from 0 to 1
fig.update_xaxes(range=[0, 1])

# sort game title by max score for gpt-4o
#fig.update_yaxes(categoryorder="total ascending")


In [292]:
runs_df[runs_df["game"] == "hollywood"]

,game,model,Max Normalized Score,Stochastic,Dialog,Darkness,Nonstandard Action,Inventory,Maze,Trivia,Self-Maintenance,Combat,Time,Mechanical Puzzle,Persistence,Difficulty,color,order
45,hollywood,meta-llama-3-70b-instruct-4,0.0,True,False,True,False,False,True,True,False,True,False,True,False,Difficult,<span style='color:orange'>hollywood</span>,1
72,hollywood,microsoft/Phi-3.5-mini-instruct,0.0,True,False,True,False,False,True,True,False,True,False,True,False,Difficult,<span style='color:orange'>hollywood</span>,1
22,hollywood,phi-3-medium-128k-instruct-1,0.0,True,False,True,False,False,True,True,False,True,False,True,False,Difficult,<span style='color:orange'>hollywood</span>,1


In [ ]:
# Create a group bar chart with the average score per game for each model
import plotly.express as px
import plotly.graph_objects as go

# sort by max score for gpt-4o
#runs_df = runs_df.sort_values(by=["model", "Max Normalized Score"], ascending=True)
# Strip the extension from game name
def color(text, diff):
    if diff == "Possible":
        s = '$\color{green}{' + str(text) + '}$'
    elif diff == "Difficult":
        s = '$\color{yellow}{' + str(text) + '}$'
    elif diff == "Extreme":
        s = '$\color{red}{' + str(text) + '}$'

    return s

fig.update_yaxes(tickvals=runs_df["game"], tickmode="array", ticktext=runs_df[["game", "Difficulty"]].map(lambda : color(game, diff)))
